## ECG Data Compression Wavelet Transform Method
Based on "Effective high compression of ECG signals at low level distortion" by Laura Rebollo-Neira

### Step 0: Import Libraries

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import iqr
import sys
import copy
from scipy.signal import detrend
import copy 

### Load Data

In [1]:
import mne
file = "/Users/juseo\Documents\dyuku\summer 20\dbdp\code\DBDP_Compression_Toolbox/Joe/ECG.EDF"
data = mne.io.read_raw_edf(file)
raw_data = data.get_data()

import numpy as np
header = ','.join(data.ch_names)
np.savetxt('ECG.csv', data.get_data().T, delimiter=',', header=header)

import pandas as pd
df0 = pd.read_csv('ECG.csv')
df = df0.iloc[:,0:1]

Extracting EDF parameters from C:\Users\juseo\Documents\dyuku\summer 20\dbdp\code\DBDP_Compression_Toolbox\Joe\ECG.EDF...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [2]:
df

,# ECG
0,0.032767
1,0.032767
2,0.032767
3,0.032767
4,0.032767
5,0.032767
6,0.032767
7,0.032767
8,0.032767
9,0.032767


### Step 1: Approximation
**Input:** Array **w** of wavelets coefficients & parameter *tol* for the approximation error  
**Output:** Indices *l*<sub>i</sub>, i=1,...,*K* of the selected wavelet coefficients. Array **c** with the selected wavelet coefficients.

#### Part A: DWT transform
Array *data* with the N-dimensional signal. Decomposition level *lv*.  
Wavelet used: bior4.4.

In [8]:
import pywt
from pywt import wavedec
def dwt(data, lv):
    cA5, cD5, cD4, cD3, cD2, cD1 = wavedec(data, 'bior4.4', level=lv)
    coeffs = {'cA5': cA5, 'cD5': cD5, 'cD4': cD4, 'cD3': cD3, 'cD2': cD2, 'cD1': cD1}
    return coeffs

In [11]:
coeffs = dwt(df,5)

Wavelet Object: https://pywavelets.readthedocs.io/en/latest/regression/wavelet.html  
DWT API: https://pywavelets.readthedocs.io/en/latest/ref/dwt-discrete-wavelet-transform.html#ref-dwt   
Handling DWT Coeffs: https://pywavelets.readthedocs.io/en/latest/ref/dwt-coefficient-handling.html

#### Part B: Selection of the largest wavelet coefficients

Thresholds the wavelet coefficients to a specified energy.  
Different levels of decomposition are thresholded at different energy percentages.
1. Calculate the energy of all the coefficients.
1. Sort the coefficients from highest to lowest.
1. Calculate the energy as each new coefficient is added.
1. Repeat until the percentage is above the threshold.

In [22]:
#sort the absolute value of the coefficients in descending order
tmp_coeffs = np.sort(np.abs(coeffs['cA5']))

In [23]:
tmp_coeffs

array([[0.18535814, 0.18535814, 0.18535814, ..., 0.18535814, 0.18535814,
        0.18535814],
       [0.18535814, 0.18535814, 0.18535814, ..., 0.18535814, 0.18535814,
        0.18535814],
       [0.18535814, 0.18535814, 0.18535814, ..., 0.18535814, 0.18535814,
        0.18535814],
       ...,
       [0.00976373, 0.00976373, 0.00976373, ..., 0.00976373, 0.00976373,
        0.00976373],
       [0.00794222, 0.00794222, 0.00794222, ..., 0.00794222, 0.00794222,
        0.00794222],
       [0.0064771 , 0.0064771 , 0.0064771 , ..., 0.0064771 , 0.0064771 ,
        0.0064771 ]])

In [49]:
print(len(tmp_coeffs))
print(len(coeffs['cA5']))
print(coeffs['cA5'][3269999])

3270000
3270000
[0.0064771 0.0064771 0.0064771 0.0064771 0.0064771 0.0064771 0.0064771
 0.0064771]


In [24]:
#find cumulative sum of square of each value
csums = np.cumsum(np.square(tmp_coeffs))

In [25]:
csums

array([3.43576412e-02, 6.87152825e-02, 1.03072924e-01, ...,
       1.07990080e+04, 1.07990080e+04, 1.07990081e+04])

In [27]:
#approximation error parameter
tol = 1

In [29]:
#set any coefficients below the threshold to zero
inds_to_zero = np.where((csums < tol))[0]
csums[inds_to_zero] = 0

In [30]:
csums

array([    0.        ,     0.        ,     0.        , ...,
       10799.00797945, 10799.0080214 , 10799.00806336])

In [31]:
nonzero_coeff_count = len(csums)

In [32]:
nonzero_coeff_count

26160000

In [33]:
#create the binary map
binary_map_tmp = np.ones(len(df)).astype(int)
binary_map_tmp[inds_to_zero] = 0

In [34]:
binary_map_tmp

array([0, 0, 0, ..., 1, 1, 1])

In [35]:
np.nonzero(csums)

(array([      29,       30,       31, ..., 26159997, 26159998, 26159999],
       dtype=int64),)

In [43]:
np.argwhere(csums > tol)

array([[      29],
       [      30],
       [      31],
       ...,
       [26159997],
       [26159998],
       [26159999]], dtype=int64)

In [44]:
coeffs['cA5'][0]

array([0.18535814, 0.18535814, 0.18535814, 0.18535814, 0.18535814,
       0.18535814, 0.18535814, 0.18535814])

NameError: name 'energy' is not defined

In [50]:
select = coeffs['cA5'][np.argwhere(csums > tol)]

IndexError: index 3270000 is out of bounds for axis 0 with size 3270000

In [ ]:
def threshold_energy(coeffs):
    #make a deep copy of coeffs to retain the original version
    coeffs_orig = copy.deepcopy(coeffs)

    binary_map = {}
    nonzero_coeff_count = {}

    for key in coeffs.keys():
        #sort the absolute value of the coefficients in ascending order
        tmp_coeffs = np.sort(np.abs(coeffs[key]))

        #calculate the threshold for retaining some percentage of the energy
        if key == 'cA5':
            thresh_perc = THRESH_PERC_APPROX
        elif key == 'cD5':
            thresh_perc = THRESH_PERC_D5
        else:
            thresh_perc = THRESH_PERC_D4_D1

        energy_thresholded = thresh_perc*energy(tmp_coeffs)
        energy_tmp = 0
        for coeff in tmp_coeffs:
            energy_tmp = energy_tmp + coeff**2

            if energy_tmp >= energy_thresholded:
                threshold = coeff
                break

        #set any coefficients below the threshold to zero
        tmp_coeffs = coeffs[key]
        inds_to_zero = np.where((tmp_coeffs < threshold) & (tmp_coeffs > -threshold))[0]
        tmp_coeffs[inds_to_zero] = 0
        
        #create array of selected wavelet coefficients
        coeffs[key] = tmp_coeffs
        nonzero_coeff_count[key] = len(tmp_coeffs)
        
    return nonzero_coeff_count   

### Step 2: Quantization
A mid-tread uniform quantizer transforms the selected wavelet coefficients into integers.

Scale the wavelet coefficients to the [0,1] range.

In [ ]:
def scale_coeffs(coeffs, do_plot=False):
    coeffs_scaled = {}
    scaling_factors = {}

    for key in coeffs.keys():
        shift_factor = np.min(coeffs[key])
        coeffs_tmp = coeffs[key]-shift_factor

        scale_factor = np.max(coeffs_tmp)
        coeffs_tmp = coeffs_tmp/scale_factor

        scaling_factors[key] = {'shift_factor': shift_factor, 'scale_factor': scale_factor}
        coeffs_scaled[key] = coeffs_tmp
    return coeffs_scaled, scaling_factors

Calculate the lowest possible number of bits to quantize the wavelet coefficients such that the PRD is above the threshold.
1. Quantize the signal starting at 8 bits
1. Unquantize and reconstruct the signal
1. Calculate the PRD.
1. Repeat with 1 fewer bit (ie, 7 bits), ...

In [ ]:
def calculate_num_bits(orig_sig, coeffs_scaled, binary_map, scaling_factors, do_plot=False):
    #starting at 8 bits, keep decreasing the number of bits in the quantization
    #until the PRD is above some threshold
    num_bits = 9

    #initialize PRD to 0 so the while loop can run
    PRD = 0

    #keep track of PRD per number of bits
    PRD_dict = {}
    
    while (num_bits >= 5) and (PRD <= MAX_PRD):
        #decrement the number of bits
        num_bits = num_bits-1

        coeffs_quantized = do_quantization(coeffs_scaled, num_bits)

        #rescale the coefficients
        coeffs_unscaled = unscale_coeffs(None, coeffs_quantized, scaling_factors, num_bits)

        #do the inverse dwt
        data_reconstructed = wavelet_reconstruction(coeffs_unscaled, None, None)

        #calculate PRD
        PRD = calculate_PRD(orig_sig, data_reconstructed)
        PRD_dict[num_bits] = PRD

        #plot the reconstructed signals 
        if do_plot:
            if PRD <= MAX_PRD:
                plt.plot(t, data_reconstructed, label='Reconstructed @ %i Bits, PRD = %.2f' % (num_bits, PRD))

    #if we went over the PRD, go back up by one bit
    if PRD > MAX_PRD:
        num_bits = num_bits+1
        PRD = PRD_dict[num_bits]
    
    return num_bits, PRD

Quantization:
1. Take in the scaled wavelet coefficients
1. Multiply by 2^(num bits)
1. Round to the nearest integer.

In [ ]:
def do_quantization(coeffs, bits, do_plot=False):
    quantized_coeffs = {}

    for key in coeffs.keys():
        sig = coeffs[key]
        sig = sig*(2**bits-1)
        sig = np.round(sig)
        sig = np.array(sig).astype(int)

        quantized_coeffs[key] = sig
        
    return quantized_coeffs

### Step 3: Organization and Storage Step
1. Re-order indices in ascending order. This re-orders coefficients and corresponding signs.
1. Store re-ordered indices as smaller positive numbers by taking differences between two consecutive values. 
1. Save the size of the singal, the quantization parameter, and the arrays (reordered coefficients, signs, and indices) in HDF5 format.

#### Reference MATLAB Code

In [ ]:
#
clear 
# Options
baseline=0 # does not modify raw data
mywname='cdf97' # wavelet family
wlevel=4 # decomposition level
store=0 # Change to 1 to add Huffman entropy coding

dat=input('Please enter record number (from the MIT_BIHA_database) \n ','s');
icase=input('Please enter approximation method (a or b) \n','s');
if icase=='a'
PRD0=input('Please enter PRD0 (e.g. 0.4217)  \n');
end

del=input('Please enter quantization parameter (e.g. 32 for a) and 39 for b) \n');
%

filenamer=['ECG_Compression/MIT_BIHA_DATA/Record_' char(strcat(dat, '_11bits.dat'))];
filenamew_tmp='ECG_Compression/Comp_Record.mat'
fid=fopen(filenamer,'r');
f=fread(fid,'ubit11');
fclose(fid);

f=f-baseline;
tic
base=0;
[fw,wl]=mydwt(f,wlevel,mywname);
L=numel(fw);
if icase=='a'
tol=PRD0*norm(f)/100;
[cs,wind]=SLWC(fw,tol);
end
if icase=='b'
cs=fw;
wind=1:numel(fw);
wind=wind';
end

# quantization of wavelet coefficients
csq=sign(cs).*floor(abs(cs)/del+0.5);
I0=find(csq==0);
if isempty(I0)==0 # remove the zeros
csq(I0)=[];
wind(I0)=[];
end

# Organization and Storage
clear vc vi vs
[or,ior]=sort(wind); # sort the indices
vc=abs(csq(ior)); 

# take differences
vi=[or(1) diff(or)'];
# invert with cumsum([vi]);
# vi=vdif(or); # take differences
vs=(sign(csq(ior))+ 1)./2; # encode the signs as zeros and ones

if store==0;
save (filenamew_tmp, 'del','L', 'vi', 'vc', 'vs'); # save in HDF5 format
else
xS{1}=vs;
xC{1}=vi;
xCC{1}=vc;
ys=Huff06(xS);
yc=Huff06(xCC);
yi=Huff06(xC);
save (filenamew_tmp, 'del','L','yi','yc','ys'); # save in HDF5 format
end
hf11=dir(filenamer);
hfH=dir(filenamew_tmp);
CR=hf11.bytes/hfH.bytes #calculate CR
toc

## Data Reconstruction

### Wavelet Reconstruction

In [ ]:
def dwt_reconstruct(coeffs):
    reconstructed = pywt.waverec([coeffs['cA5'], coeffs['cD5'], coeffs['cD4'], coeffs['cD3'], 
                                    coeffs['cD2'], coeffs['cD1']], 'bior4.4')
    return reconstructed